## Homework 09:  Parallel Programming 02

## Due Date: Friday Apr 23, 2021, 05:00pm

#### Firstname Lastname: Yuhan Liu

#### E-mail: yl7576@nyu.edu

#### Enter your solutions and submit this notebook

---

**Problem 1 (40p)**

In this problem the goal is to calculate the mean and standard deviation of a large list of numbers by using Reduction as one of Collective Operations, see Lecture 11. 


Consider $N = 256000$ random variables uniform on $[0, 1]$, call these $x_0, x_1, \dots, x_{N - 1}$.  


Write an MPI program with $N=16$ processes that outputs the average and standard deviation of $x_0, x_1, \dots, x_{N - 1}$.


To simplify the problem, let one process, say `Process 0`, independently draws $N$ samples uniformly on $[0, 1]$.

How do you explain the results?


**Instructions:** 
Your program should use MPI4PY and collective operations. 
Save your program as 2020_spring_sol09_pr01.py and run it from the terminal as:

```
mpirun -n 16 python 2020_spring_sol09_pr01.py
```


In [29]:
%%writefile 2020_spring_sol09_pr01.py
import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
status = MPI.Status()

n = 256000

if rank == 0:
    x = np.random.uniform(0,1,n).astype('f').reshape(size, int(n/size))
else:
    x = None

recvbuf = np.empty(int(n/size),dtype='f')
comm.Scatter(x, recvbuf, root=0)
rank_sum = sum(recvbuf)

overall_sum = comm.allreduce(rank_sum, MPI.SUM)
overall_mean = overall_sum/n

rank_sqdiff = sum((i-overall_mean)**2 for i in recvbuf)
std = comm.reduce(rank_sqdiff, MPI.SUM, 0)

if rank==0:
    overall_std = np.sqrt(std/n)
    print('Mean is',overall_mean)
    print('Standard Deviation is', overall_std)

Overwriting 2020_spring_sol09_pr01.py


In [30]:
!mpirun -n 16 python 2020_spring_sol09_pr01.py

Mean is 0.4991937337077721
Standard Deviation is 0.2883587150140864



---
**Problem 2 (60p)**

In this problem the goal is to demonstrate how one can use a Domain Decomposition and  Collective Operations. 

Consider the exponential distribution $X \sim \textrm{Exp}(1)$ with the unit mean. Find numerical approximations of moments of the exponential random variable. 

That is, for a random variable $X$ with the distribution $f(x) = e^{-x}$ for $x \geq 0$, compute the first $15$ moments, where the $k$-th moment is defined as:
$$I_k = \int_{0}^{\infty} x^k f(x) dx.$$


Your program should use MPI parallel collective instructions, where the integration is performed in parallel over $N=16$ processes, over a finite range $[0, M)$, where $M=1000$, with $N = 16$ partitions and $1000$ increments per partition, see Lecture 10 and 11.

Provide evaluations of $J_1, J_2, \dots, J_{15}$, where $$J_k = \int_{0}^{M} x^k f(x) dx.$$


**Instructions:** 

Save your program as 2020_sol09_pr02.py; and run it from the terminal as:

```
mpirun -n 16 python 2020_spring_sol09_pr02.py
```


**Bonus Question (10 points):** 

What is the value of $I_k$, as a function of $k$? How can it be derived?

In [52]:
%%writefile 2020_spring_sol09_pr02.py
import numpy as np
import sys
from math import exp
from mpi4py import MPI

def jk(a_r,k,n,delta):
        res = 0.0
        for i in range(n):
            x = a_r + (i+0.5) * delta
            res += (x**k) * np.exp(-x) * delta
        return res
    
for k in list(range(1, 16)):
    k = int(k)
    
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    local_intg, global_intg = np.zeros(1),np.zeros(1)
    if rank == 0:
        n = np.full(1, 1000, dtype=int) 
    else:
        n = np.zeros(1, dtype=int)
    comm.Bcast(n, root=0)
    
    delta = 1000.0 / (n*size)  
    intg_start = rank * (n*delta)
    local_intg[0] = jk(intg_start, k, n[0], delta)
    
    comm.Reduce(local_intg, global_intg, MPI.SUM, 0)
    if rank == 0:
        print('with k =',k,'integral =', global_intg[0])

Overwriting 2020_spring_sol09_pr02.py


In [53]:
!mpirun -n 16 python 2020_spring_sol09_pr02.py

with k = 1 integral = 1.0001627047952104
with k = 2 integral = 2.0000001112238226
with k = 3 integral = 5.99999988885252
with k = 4 integral = 23.999999999771028
with k = 5 integral = 120.00000000022848
with k = 6 integral = 719.9999999999999
with k = 7 integral = 5040.000000000001
with k = 8 integral = 40320.00000000004
with k = 9 integral = 362879.9999999999
with k = 10 integral = 3628800.0000000023
with k = 11 integral = 39916799.99999997
with k = 12 integral = 479001600.0000007
with k = 13 integral = 6227020800.000004
with k = 14 integral = 87178291200.0002
with k = 15 integral = 1307674368000.0007
